This model takes 10 columns from the Airbnb dataset used in the previous Airbnb project.
It uses them for prediction of the price in dollars using a random forest regressor 

### Building a web app model that predicts the price of an accommodation 

In [6]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import pandas as pd
import seaborn as sns; sns.set(); sns.set_style('darkgrid'); 
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score

In [7]:
df = pd.read_csv(r'D:\Jupyter notebook\Projects\Air Bnb\AirBnb\Data\listings_after_cleaning.csv')
df.head(3)

,id,experiences_offered,host_id,host_response_time,host_response_rate,host_total_listings_count,London_Borough,property_type,room_type,accommodates,...,Air_conditioning,Kitchen,Parking,Pet_friendliness,Jacuzzi,Washer,Heating,TV,Swimming pool,Years_as_host
0,11551,family,43039,within an hour,100,3.0,Lambeth,Apartment,Entire home/apt,4,...,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,10.913297
1,13913,business,54730,within a few hours,70,4.0,Islington,Apartment,Private room,2,...,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,10.792829
2,15400,romantic,60302,within a few hours,100,1.0,Kensington and Chelsea,Apartment,Entire home/apt,2,...,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,10.740809


In [8]:
cols_to_del = ['id', 'experiences_offered', 'host_id', 'host_response_time',
       'host_response_rate', 'host_total_listings_count','beds',
       'bed_type', 'security_deposit', 'cleaning_fee',
       'guests_included', 'extra_people', 'minimum_nights', 'maximum_nights',
       'availability_365', 'number_of_reviews', 'review_scores_rating',
       'review_scores_accuracy', 'cancellation_policy', 'reviews_per_month',
        'Kitchen', 'Parking', 'Pet_friendliness',
       'Jacuzzi', 'Washer', 'TV', 'Years_as_host']
df.drop(cols_to_del,axis=1,inplace=True)
df.rename(columns={'Swimming pool':'Swimming_pool'})

,London_Borough,property_type,room_type,accommodates,bathrooms,bedrooms,price,WiFi,Air_conditioning,Heating,Swimming_pool
0,Lambeth,Apartment,Entire home/apt,4,1.0,1.0,88.0,1.0,0.0,1.0,0.0
1,Islington,Apartment,Private room,2,1.0,1.0,65.0,1.0,0.0,1.0,0.0
2,Kensington and Chelsea,Apartment,Entire home/apt,2,1.0,1.0,100.0,1.0,1.0,1.0,0.0
3,Westminster,Apartment,Entire home/apt,6,2.0,3.0,300.0,1.0,0.0,1.0,0.0
4,Hammersmith and Fulham,Bed and breakfast,Private room,2,1.0,1.0,150.0,1.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
87050,Bromley,Loft,Entire home/apt,2,1.5,1.0,65.0,1.0,0.0,1.0,0.0
87051,Lambeth,Apartment,Private room,3,1.0,1.0,69.0,1.0,0.0,1.0,0.0
87052,Ealing,House,Private room,2,1.5,2.0,45.0,1.0,0.0,1.0,0.0
87053,Westminster,Apartment,Entire home/apt,4,1.0,1.0,179.0,0.0,0.0,0.0,0.0


In [9]:
object_cols = list(df.select_dtypes(include='object').columns)
dt = pd.get_dummies(columns=object_cols,data=df)
dt = dt.astype('float64')

In [10]:
from sklearn.model_selection import train_test_split
train_val_data =  dt.drop('price',axis=1)[:80000]
train_val_targets = dt.price[:80000]
test_targets = dt.price[80000:]
test_data = dt.drop('price',axis=1)[80000:]
x_train,x_val,y_train,y_val = train_test_split(train_val_data,train_val_targets,shuffle=True,test_size=0.3)

In [121]:
import joblib
model_columns = list(x_train.columns)
joblib.dump(model_columns, 'model_columns.pkl')

['model_columns.pkl']

In [128]:
from sklearn.ensemble import RandomForestRegressor

In [130]:
Rfr = RandomForestRegressor(n_jobs=-1,n_estimators=500,oob_score=True,min_samples_leaf=30)
training = Rfr.fit(x_train,y_train)
val_predicted = Rfr.predict(x_val)
mae = mean_absolute_error(y_val,val_predicted)
rmse = mean_squared_error(val_predicted,y_val,squared=False)
oob_score = Rfr.oob_score_
r_sq = r2_score(y_val,val_predicted)
print('mae: ' , round(mae,2))
print('Rmse: ' , round(rmse,2))
print('oob_Score: ' ,round(oob_score,2))
print('R-squared: ' , round(r_sq,3))

mae:  55.79
Rmse:  223.07
oob_Score:  0.26
R-squared:  0.212


In [131]:
joblib.dump(Rfr,'Rfr-airbnb.pkl')

['Rfr-airbnb.pkl']

# Conclusions
* The model has a very poor R squared value but this was expected since the web app is being used for trial purposes
* The model can be improved by tuning the hyperparameters 